In [3]:
import os
import polars as pl

In [95]:
%%timeit
df = pl.read_parquet(os.path.join("data","cnb_vyber.parquet"))

890 ms ± 65.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
def najdi_rok(nn8):
    if nn8[6] in ['s', 't', 'd', 'm', 'e', 'q', 'c','u', 'r']:
        if ('x' not in nn8[7:11]) and ('u' not in nn8[7:11]):
            return int(nn8[7:11])
        else:
            return None
    else:
        return None

In [105]:
df = df.explode('008')

InvalidOperationError: `explode` operation not supported for dtype `str`

In [103]:
df.select('008')

008
str
"""010202s2000 xr a 0…"
"""930610s1948 xr e d 0…"
"""930610s1948 xr e d 0…"
"""931001s1991 xr 0…"
"""931001s1991 xr 0…"
…
"""920329s1992 xr a u0…"
"""930618s1991 xr d 0…"
"""930618s1991 xr d 0…"


In [109]:
df = df.with_columns(pl.col('008').map_elements(najdi_rok).alias('rok'))

C:\Users\micha\AppData\Local\Temp\ipykernel_23896\3959673879.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col('008').map_elements(najdi_rok).alias('rok'))


In [115]:
%%timeit
print(df.select(['245_a','rok']).sort('rok'))

shape: (957_280, 2)
┌─────────────────────────────────┬──────┐
│ 245_a                           ┆ rok  │
│ ---                             ┆ ---  │
│ list[str]                       ┆ i64  │
╞═════════════════════════════════╪══════╡
│ ["Tita Livia dějiny od založen… ┆ null │
│ ["Tita Livia dějiny od založen… ┆ null │
│ ["Pověsti z Telče a okolí /"]   ┆ null │
│ ["Testy studijních předpokladů… ┆ null │
│ ["Paličák :"]                   ┆ null │
│ …                               ┆ …    │
│ ["Kytice :"]                    ┆ 2024 │
│ ["Pět říší."]                   ┆ 2024 │
│ ["Vánoce :"]                    ┆ 2024 │
│ ["To si zkus /"]                ┆ 2024 │
│ ["Fotografova hřbitovní překva… ┆ 2025 │
└─────────────────────────────────┴──────┘
shape: (957_280, 2)
┌─────────────────────────────────┬──────┐
│ 245_a                           ┆ rok  │
│ ---                             ┆ ---  │
│ list[str]                       ┆ i64  │
╞═════════════════════════════════╪══════╡
│ ["Tita Livia

In [117]:
print(df.select(['245_a','rok']).sort('rok').drop_nulls())

shape: (957_260, 2)
┌─────────────────────────────────┬──────┐
│ 245_a                           ┆ rok  │
│ ---                             ┆ ---  │
│ list[str]                       ┆ i64  │
╞═════════════════════════════════╪══════╡
│ ["Příručka bohoslužebná pro če… ┆ 1900 │
│ ["Srovnávací přehled některých… ┆ 1900 │
│ ["Na okraj dnů :"]              ┆ 1900 │
│ ["Farma "Boží oko" :"]          ┆ 1900 │
│ ["Hloupý Jack a moudrý James :… ┆ 1900 │
│ …                               ┆ …    │
│ ["Kytice :"]                    ┆ 2024 │
│ ["Pět říší."]                   ┆ 2024 │
│ ["Vánoce :"]                    ┆ 2024 │
│ ["To si zkus /"]                ┆ 2024 │
│ ["Fotografova hřbitovní překva… ┆ 2025 │
└─────────────────────────────────┴──────┘


In [31]:
filtered_df = df.explode("245_a").filter(pl.col("245_a").str.contains("láska"))
print(filtered_df.select("245_a","100_a"))

shape: (1_517, 2)
┌─────────────────────────────┬──────────────────────────┐
│ 245_a                       ┆ 100_a                    │
│ ---                         ┆ ---                      │
│ str                         ┆ list[str]                │
╞═════════════════════════════╪══════════════════════════╡
│ Když přišla na svět láska / ┆ ["Pfannmüller, Donatus"] │
│ Prodaná láska /             ┆ ["Neruda, Jan,"]         │
│ Krása a láska /             ┆ ["Lalo, Charles,"]       │
│ Tylova láska :              ┆ ["Strejček, Ferdinand,"] │
│ Zločinná láska :            ┆ ["Braddon, M. E."]       │
│ …                           ┆ …                        │
│ Tvá láska je můj lék /      ┆ ["Carol, Ly"]            │
│ Divná láska /               ┆ ["Cooper, M. E."]        │
│ Divná láska /               ┆ ["Cooper, M. E."]        │
│ Nespoutaná láska /          ┆ ["Rainville, Rita"]      │
│ Nespoutaná láska /          ┆ ["Rainville, Rita"]      │
└─────────────────────────────┴───────